<a href="https://colab.research.google.com/github/Padiwa/machinelearnia/blob/main/Copie_de_Breast_cancer_06_11_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import random
import sys
import cv2
import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split

# Import des données

In [3]:
!git clone https://github.com/MachineLearnia/breast_cancer_public_data.git

Cloning into 'breast_cancer_public_data'...
remote: Enumerating objects: 4203, done.
remote: Counting objects: 100% (805/805), done.
remote: Compressing objects: 100% (805/805), done.
remote: Total 4203 (delta 0), reused 805 (delta 0), pack-reused 3398 (from 2)
Receiving objects: 100% (4203/4203), 90.60 MiB | 15.84 MiB/s, done.
Resolving deltas: 100% (6/6), done.


In [4]:
!ls

breast_cancer_public_data  sample_data


In [5]:
os.chdir("breast_cancer_public_data")

In [8]:
!ls

data  data_2  README.md


In [7]:
folder_path = "./data_2"

In [9]:
# Train data preparation
classes =  ["Negative","Cancer"]
dataset = []
for class_label in classes:
    class_path = os.path.join(folder_path, class_label)
    label_index = classes.index(class_label)
    for img_file in tqdm.tqdm(os.listdir(class_path)):
        img_path = os.path.join(class_path, img_file)
        img = cv2.imread(img_path)
        img = cv2.resize(img,(224,224))
        dataset.append([img, label_index])

100%|██████████| 410/410 [00:00<00:00, 1707.10it/s]


# Normalisation

In [12]:
x_images = []
y_images = []
for img, label in dataset:
  x_images.append(img)
  y_images.append(label)

x = np.array(x_images) / 255
y = np.array(y_images)


# Découpe en un jeu d'entrainement et un jeu de Test

In [13]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=42)

In [14]:
x_test.shape

(164, 224, 224, 3)

In [15]:
x_train.shape

(656, 224, 224, 3)

# Entrainement du CNN :

In [16]:
model = Sequential()

model.add(Conv2D(100,(3,3),activation='relu',input_shape=(224,224,3)))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(100,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))


model.add(Conv2D(100,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))


model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1,activation="sigmoid"))


model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 100)  │         2,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 100)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 100)  │        90,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 100)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 100)    │        90,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 100)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 67600)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │     2,163,232 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,346,265 (8.95 MB)

 Trainable params: 2,346,265 (8.95 MB)

 Non-trainable params: 0 (0.00 B)

In [17]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [18]:
# Training the model
history = model.fit(x_train, y_train, epochs=30, validation_batch_size=(x_test, y_test))

Epoch 1/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 20s 400ms/step - accuracy: 0.5547 - loss: 0.6805
Epoch 2/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 3s 122ms/step - accuracy: 0.8732 - loss: 0.4125
Epoch 3/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 3s 124ms/step - accuracy: 0.8757 - loss: 0.2854
Epoch 4/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 3s 125ms/step - accuracy: 0.9351 - loss: 0.1905
Epoch 5/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 3s 123ms/step - accuracy: 0.9453 - loss: 0.1473
Epoch 6/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 3s 123ms/step - accuracy: 0.9617 - loss: 0.1210
Epoch 7/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 3s 124ms/step - accuracy: 0.9828 - loss: 0.0617
Epoch 8/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 3s 127ms/step - accuracy: 0.9745 - loss: 0.0656
Epoch 9/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 3s 129ms/step - accuracy: 0.9833 - loss: 0.0433
Epoch 10/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 3s 126ms/step - accuracy: 0.9888 - loss: 0.0316
Epoch 11/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 3s 125ms/step - accuracy: 0.9828 - loss: 0.0441
Epoch 12/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 3s 147ms/ste

# Visualisation des résultats